In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

from numpy.polynomial import polynomial as P
import numpy as np

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=False)

In [ ]:
class JMTSolver(object):
    def __init__(self, a, b, v1, v2, t):
        self.c = [
            a,
            v1,
            0,
            -2 * (5*a - 5*b + 3*t*v1 + 2*t*v2) / t**3,
            (15*a - 15*b + 8*t*v1 + 7*t*v2) / t**4,
            -3 * (2*a - 2*b + t*v1 + t*v2) / t**5
        ]
                
    def f(self, x):
        return P.polyval(x, self.c)
    
    def fd(self, x):
        return P.polyval(x, P.polyder(self.c, 1))
    
    def fdd(self, x):
        return P.polyval(x, P.polyder(self.c, 2))

class LinearSolver(object):
    def __init__(self, a, b, t):
        assert(t > 0)
        self.c0 = a
        self.c1 = (b - a) / t
        
    def f(self, x):
        return self.c0 + self.c1 * x
    
    def fd(self, x):
        return self.c1
    
    def fdd(self, x):
        return 0
        
        
def curvature(solverx, solvery, t):
    return ((solverx.fd(t) * solvery.fdd(t) - solverx.fdd(t) * solvery.fd(t)) /
            (solverx.fd(t) ** 2 + solvery.fd(t) ** 2) ** 1.5)
        

T = 10
solverx = LinearSolver(0.0, 40, T)
solvery = JMTSolver(0.0, 4.0, 0, 0, T)
        
L = 100
t = np.arange(0, L + 1, dtype=np.float)
x = solverx.f(t / L * T)
y = solvery.f(t / L * T)
c = curvature(solverx, solvery, t)

data = []
data.append(go.Scatter(
    x=x,
    y=y,
    mode="lines+markers",
    text=list(map(str, c))
))

layout = go.Layout(
    height=900,
    yaxis={
        "scaleanchor": "x",
    },
)

figure = go.Figure(data=data, layout=layout)
iplot(figure, filename="JMT")

In [ ]:
obstacles = [[31, 0.1, 49, 1.9]]

width = 6
length = 100
x_step = 10


class Edge(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def is_ok(self, obstacles):
        for obstacle in obstacles:
            for (x, y) in zip(self.x, self.y):
                if (obstacle[0] <= x <= obstacle[2] and obstacle[1] <= y <= obstacle[3]):
                    return False
        return True

In [ ]:

def generate_edge(start_x, start_y, end_x, end_y):
    if (abs(start_y - end_y) < 1e-3):
        solver = LinearSolver(start_y, end_y, end_x - start_x)
    else:
        solver = JMTSolver(start_y, end_y, 0, 0, end_x - start_x)
    xs = np.arange(start_x, end_x + 1e-3, (end_x - start_x) / 10)
    ys = solver.f(xs - start_x)
    return Edge(xs, ys)


finish_x = 100
finish_y = 1
    
def dfs(start_x, start_y, obstacles):
    if start_x > length:
        return []
    if start_y < 0 or start_y > width:
        return []
    for y in [0, -1, 1]:
        e = generate_edge(start_x, start_y, start_x + x_step, start_y + y)
        if not e.is_ok(obstacles):
            continue
        if (start_x == finish_x and start_y + y == finish_y):
            return [e]
        result = dfs(start_x + x_step, start_y + y, obstacles)
        if not result:
            continue
        return [e] + result
    return []


list(map(lambda e: "{},{}->{},{}".format(int(e.x[0]), int(e.y[0]), int(e.x[-1]), int(e.y[-1])), dfs(0, 1, obstacles)))

In [ ]:
shapes = []

road =  {
    'type': 'rect',
    'x0': 0,
    'y0': 0,
    'x1': length,
    'y1': width,
    'fillcolor': 'rgba(0, 0, 128, 0.3)',
}

shapes.append(road)


for obstacle in obstacles:
    shapes.append({
        'opacity': 0.7,
        'xref': 'x',
        'yref': 'y',
        'fillcolor': 'grey',
        'x0': obstacle[0],
        'y0': obstacle[1],
        'x1': obstacle[2],
        'y1': obstacle[3],
        'type': 'circle',
        'line': {
            'color': 'grey',
        },
    })
   

edges = []
for ms in range(0, length - 1, x_step):
    for y in range(0, width + 1, 1):
        edges.append(generate_edge(ms, y, ms + x_step, y))
        
        if (y > 0):
            edges.append(generate_edge(ms, y, ms + x_step, y - 1))
            
        if (y < width):
            edges.append(generate_edge(ms, y, ms + x_step, y + 1))
            
            
edges = list(filter(lambda x: x.is_ok(obstacles), edges))            
            
data = []
for ms in range(0, length + 1, x_step):
    x = [ms] * (width + 1)
    y = list(range(width + 1))
    data.append(go.Scatter(
        x=x, y=y, 
        mode="lines",
        marker=dict(color='green'), 
        showlegend=False,
        hoverinfo='none'))
    

for edge in edges:
    data.append(go.Scatter(
        x=edge.x,
        y=edge.y,
        mode="lines+markers",
        marker=dict(color='orange'),
        showlegend=False,
        hoverinfo='none'))


iplot(dict(data=data, layout={'shapes': shapes}), filename='fig')